In [4]:
import pandas as pd
import numpy as np

**READING DATA USING PANDAS**

We use pandas read_csv function to tead the csv file in python and pandas dataframe method to convert file into the data frame.

In [5]:
df=pd.DataFrame(pd.read_csv("/content/train.csv"))#read_csv is the function(of pandas module) is used to read csv file
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.shape

(891, 12)

In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

**Seperating out the columns which have more than 35% of the values missing in the dataset**

In [8]:
x=df.isnull().sum()
drop_col=x[x>(35/100*df.shape[0])]
drop_col

Cabin    687
dtype: int64

In [9]:
drop_col.index

Index(['Cabin'], dtype='object')

**as there are many null values in the cabin column so we are dropping the column using drop method.**

In [10]:
df.drop(drop_col.index,axis=1,inplace=True)#axis=1 , here 1 is for column.
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

as there are 2 more columns with some null values so we are filling those null values by the mean of the values already present in the column.

In [11]:
df.fillna(df.mean(),inplace=True)
df.isnull().sum()# embarked column still have 2 empty rows , because it contains string data and its mean cannot be taken.

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

**printing all the details of column "Embarked" using describe function.**

In [12]:
df["Embarked"].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

**count**=non-null values
**unique**=unique values
**top**=value having the highest frequency
**freq**=highest frequency

**For Embarked attribute,we fill the NULL values with the most frequent value in the column.**

In [13]:
df["Embarked"].fillna("S",inplace=True)


In [14]:
df.isnull().sum()#now all the null values have been filled

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

**we can see the correlation is maximum between fare and pclass**

In [15]:
df.corr()#corr() returns the correlation(proportionality) between values of 2 columns

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.033207,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500
Age,0.033207,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566
SibSp,-0.057527,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000


**sibsp**=no. of siblings/spouses aboard

**parch**=no. of parents/children aboard

**so we can make a new column family_size by combining these 2 columns.**

In [16]:
df["Family_Size"]=df["SibSp"]+df["Parch"]
df.drop(["SibSp","Parch"],axis=1,inplace=True)
df.corr()

,PassengerId,Survived,Pclass,Age,Fare,Family_Size
PassengerId,1.000000,-0.005007,-0.035144,0.033207,0.012658,-0.040143
Survived,-0.005007,1.000000,-0.338481,-0.069809,0.257307,0.016639
Pclass,-0.035144,-0.338481,1.000000,-0.331339,-0.549500,0.065997
Age,0.033207,-0.069809,-0.331339,1.000000,0.091566,-0.248512
Fare,0.012658,0.257307,-0.549500,0.091566,1.000000,0.217138
Family_Size,-0.040143,0.016639,0.065997,-0.248512,0.217138,1.000000


we ab see that , familysize in the ship does not have much correlance with survival rate.


**Now lets check whether the person was alone or not affects the survival rate**

In [17]:
df["Alone"]=[0 if df["Family_Size"][i]>0 else 1 for i in df.index]#it will store the value 0 if not alone and ,  if  alone then 1.
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,Family_Size,Alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,S,0,1


In [18]:
df.groupby(["Alone"])["Survived"].mean()

Alone
0    0.505650
1    0.303538
Name: Survived, dtype: float64

it means that here we can see that , if the person is alone has less chance of surviving.

**Correlation between 2 columns**

In [19]:
df[["Alone","Fare"]].corr()

,Alone,Fare
Alone,1.000000,-0.271832
Fare,-0.271832,1.000000


now we can see that , if the person was not alone the chance the ticket price is higher is high

In [20]:
df["Sex"]=[0 if df["Sex"][i] == "male" else 1 for i in df.index]
df.groupby(["Sex"])["Survived"].mean()

Sex
0    0.188908
1    0.742038
Name: Survived, dtype: float64

it shows that , female passengers have more chance of surviving than male ones.

In [21]:
df.groupby(["Embarked"])["Survived"].mean()

Embarked
C    0.553571
Q    0.389610
S    0.339009
Name: Survived, dtype: float64

it shows that people who embarked with C(Cherbourg) have higher Survival Rate.

In [22]:
df.groupby(["Age"])["Survived"].mean()
df[["Age","Survived"]].corr()

,Age,Survived
Age,1.000000,-0.069809
Survived,-0.069809,1.000000


In [31]:
df.groupby(["Pclass"])["PassengerId"].count()

Pclass
1    216
2    184
3    491
Name: PassengerId, dtype: int64

In [37]:
df.groupby(["Survived","Pclass"])["Pclass"].count()

Survived  Pclass
0         1          80
          2          97
          3         372
1         1         136
          2          87
          3         119
Name: Pclass, dtype: int64

**CONCLUSION :-**

*   Number of Passengers travelling in 3rd passenger's is highest.

*   Passengers travelling in 1st Passenger's class have survived more.

*   Female passengers were prioritized over men.

*   People with high class or rich people have higher survival rate than others. The hierarichy might have been followed while saving the passengers.

*   Passengers travelling with their family have higher survival rate.

*   Passengers travelling in 3rd Passenger's class has died most.

*   Passengers who borded the ship ar Cherbourg, survived more in proportion then the others.

*   Passengers travelling in 2nd Passenger's class have survived least.